<a href="https://colab.research.google.com/github/DhruvaBansal00/CV-Spring2020-Project/blob/main/baseline_EfficientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

import tensorflow_hub as hub

import tensorflow_datasets as tfds


In [13]:
# Import dataset data from tfds

NUM_CLASSES = 10
dataset_name = 'cifar'+str(NUM_CLASSES)
ds_train, info = tfds.load(name=dataset_name, split='train', with_info=True)
ds_test = tfds.load(name=dataset_name, split='test')
train_examples = info.splits['train'].num_examples

In [14]:
# Split into train and test sets
# We have checked that the classes are reasonably balanced.
# train_split = 0.98
# num_train = int(train_split * num_examples)
# ds_train = ds_train.take(num_train)
DATASET_NUM_TRAIN_EXAMPLES = train_examples

In [15]:
# Load model from TFHub into KerasLayer
efficient_net_url = "https://tfhub.dev/google/efficientnet/b3/feature-vector/1"
efficientNetKeras = hub.KerasLayer(efficient_net_url)

In [16]:
class EfficientNetModel(tf.keras.Model):
  """Efficient Net with a new head."""

  def __init__(self, num_classes, module):
    super().__init__()

    self.num_classes = num_classes
    self.head = tf.keras.layers.Dense(num_classes, kernel_initializer='zeros')
    self.bit_model = module
  
  def call(self, images):
    # No need to cut head off since we are using feature extractor model
    bit_embedding = self.bit_model(images)
    return self.head(bit_embedding)

model = EfficientNetModel(num_classes=NUM_CLASSES, module=efficientNetKeras)

In [18]:

# Preprocessing helper functions
RESIZE_TO = 160
CROP_TO = 128
# Create data pipelines for training and testing:
BATCH_SIZE = 512
SCHEDULE_LENGTH = 10000
SCHEDULE_LENGTH = SCHEDULE_LENGTH * 512 / BATCH_SIZE

STEPS_PER_EPOCH = 10

def cast_to_tuple(features):
  return (features['image'], features['label'])
  
def preprocess_train(features):
  # Apply random crops and horizontal flips for all tasks 
  # except those for which cropping or flipping destroys the label semantics
  # (e.g. predict orientation of an object)
  features['image'] = tf.image.random_flip_left_right(features['image'])
  features['image'] = tf.image.resize(features['image'], [RESIZE_TO, RESIZE_TO])
  features['image'] = tf.image.random_crop(features['image'], [CROP_TO, CROP_TO, 3])
  features['image'] = tf.cast(features['image'], tf.float32) / 255.0
  return features

def preprocess_test(features):
  features['image'] = tf.image.resize(features['image'], [RESIZE_TO, RESIZE_TO])
  features['image'] = tf.cast(features['image'], tf.float32) / 255.0
  return features

pipeline_train = (ds_train
                  .shuffle(200000)
                  .repeat(int(SCHEDULE_LENGTH * BATCH_SIZE / DATASET_NUM_TRAIN_EXAMPLES * STEPS_PER_EPOCH) + 1 + 50)  # repeat dataset_size / num_steps
                  .map(preprocess_train, num_parallel_calls=8)
                  .batch(BATCH_SIZE)
                  .map(cast_to_tuple)  # for keras model.fit
                  .prefetch(2))

pipeline_test = (ds_test.map(preprocess_test, num_parallel_calls=1)
                  .map(cast_to_tuple)  # for keras model.fit
                  .batch(BATCH_SIZE)
                  .prefetch(2))

In [19]:
 SCHEDULE_BOUNDARIES = [3000, 6000, 9000]
lr = 0.003 * BATCH_SIZE / 512 
# Decay learning rate by a factor of 10 at SCHEDULE_BOUNDARIES.
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries=SCHEDULE_BOUNDARIES, 
                                                                   values=[lr, lr*0.1, lr*0.001, lr*0.0001])
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [20]:
model.compile(optimizer=optimizer,
              loss=loss_fn,
              metrics=['accuracy'])

# Fine-tune model
history = model.fit(
    pipeline_train,
    batch_size=BATCH_SIZE,
    steps_per_epoch=STEPS_PER_EPOCH,
    epochs= int(SCHEDULE_LENGTH / STEPS_PER_EPOCH), 
    validation_data=pipeline_test  
)

Epoch 1/1000
10/10 [==============================] - 25s 3s/step - loss: 1.3589 - accuracy: 0.7070 - val_loss: 0.6364 - val_accuracy: 0.8516
Epoch 2/1000
10/10 [==============================] - 21s 2s/step - loss: 0.6450 - accuracy: 0.8025 - val_loss: 0.4426 - val_accuracy: 0.8638
Epoch 3/1000
10/10 [==============================] - 20s 2s/step - loss: 0.5807 - accuracy: 0.8082 - val_loss: 0.3990 - val_accuracy: 0.8712
Epoch 4/1000
10/10 [==============================] - 20s 2s/step - loss: 0.5186 - accuracy: 0.8244 - val_loss: 0.3762 - val_accuracy: 0.8763
Epoch 5/1000
10/10 [==============================] - 20s 2s/step - loss: 0.4883 - accuracy: 0.8352 - val_loss: 0.3572 - val_accuracy: 0.8838
Epoch 6/1000
10/10 [==============================] - 21s 2s/step - loss: 0.4485 - accuracy: 0.8463 - val_loss: 0.3495 - val_accuracy: 0.8838
Epoch 7/1000
10/10 [==============================] - 20s 2s/step - loss: 0.4967 - accuracy: 0.8354 - val_loss: 0.3404 - val_accuracy: 0.8879
Epoch 

KeyboardInterrupt: ignored